Import some libraries we'll use:

In [1]:
import urllib.request
import gzip
import pandas as pd
import os

Download the data from NOAA. The columns in the file are documented [here](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/readme.txt).

In [2]:
file = '1975.csv.gz'
if os.path.isfile(file):
    print(file, 'already downloaded.')
else:
    urllib.request.urlretrieve('https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/1975.csv.gz', file)

1975.csv.gz already downloaded.


Before we can read the file, we'll define our own date parser:

In [3]:
noaadateparser = lambda x: pd.datetime.strptime(x, "%Y%m%d")


Unzip the file and load the CSV with pandas, using our date parser. We'll also give the columns names since the CSV doesn't have a header 🧟‍:

In [4]:
with gzip.open('1975.csv.gz') as f:

    data_1975 = pd.read_csv(f, 
                            names = ["station", "date", "type", "value"],
                            usecols = [0,1,2,3],
                            parse_dates = ["date"],
                            date_parser = noaadateparser)

data_1975.head()

,station,date,type,value
0,CA008203164,1975-01-01,TMAX,17
1,CA008203164,1975-01-01,TMIN,-11
2,CA008203164,1975-01-01,PRCP,216
3,CA008203164,1975-01-01,SNOW,216
4,CA008105560,1975-01-01,TMAX,17


In [5]:
data_1975.dtypes

station            object
date       datetime64[ns]
type               object
value               int64
dtype: object

Only keep TMIN and TMAX:

In [6]:
data_1975 = data_1975[(data_1975["type"] == "TMAX") | (data_1975["type"] == "TMIN")] 

In [7]:
data_1975.head

<bound method NDFrame.head of               station       date  type  value
0         CA008203164 1975-01-01  TMAX     17
1         CA008203164 1975-01-01  TMIN    -11
4         CA008105560 1975-01-01  TMAX     17
5         CA008105560 1975-01-01  TMIN    -67
10        CA007056600 1975-01-01  TMAX    -22
11        CA007056600 1975-01-01  TMIN   -111
15        CA007048421 1975-01-01  TMAX   -100
16        CA007048421 1975-01-01  TMIN   -278
19        CA007045400 1975-01-01  TMAX   -139
20        CA007045400 1975-01-01  TMIN   -261
24        ASN00040101 1975-01-01  TMAX    336
26        CA007020860 1975-01-01  TMAX    -11
27        CA007020860 1975-01-01  TMIN    -89
31        CA006166450 1975-01-01  TMAX     11
32        CA006166450 1975-01-01  TMIN    -39
35        CA006141919 1975-01-01  TMAX    -17
36        CA006141919 1975-01-01  TMIN    -61
40        CA006107955 1975-01-01  TMAX      0
41        CA006107955 1975-01-01  TMIN    -50
45        ASN00014814 1975-01-01  TMAX    386
46  

Next, let's get the list of stations so that we have lat/lon for each:

In [8]:
stations = pd.read_fwf("https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt", 
            infer_nrows=300, # how many rows to use to infer the column widths
            usecols = [0,1,2,3,4],
            names = ["station", "lat", "lon", "elevation", "name"])

stations.head()

,station,lat,lon,elevation,name
0,ACW00011604,17.1167,-61.7833,10.1,ST JOHNS COOLIDGE FLD
1,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS
2,AE000041196,25.3330,55.5170,34.0,SHARJAH INTER. AIRP
3,AEM00041194,25.2550,55.3640,10.4,DUBAI INTL
4,AEM00041217,24.4330,54.6510,26.8,ABU DHABI INTL


Save the stations as a "proper" CSV:

In [16]:
stations.to_csv('stations.csv')

Pull out the country ID from the station column (first two letters):

In [9]:
stations["country"] = stations["station"].astype(str).str[0:2]
stations.head()

,station,lat,lon,elevation,name,country
0,ACW00011604,17.1167,-61.7833,10.1,ST JOHNS COOLIDGE FLD,AC
1,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,AC
2,AE000041196,25.3330,55.5170,34.0,SHARJAH INTER. AIRP,AE
3,AEM00041194,25.2550,55.3640,10.4,DUBAI INTL,AE
4,AEM00041217,24.4330,54.6510,26.8,ABU DHABI INTL,AE


Join the two together:

In [10]:
data_1975 = pd.merge(data_1975, stations, on="station")
data_1975.head()



,station,date,type,value,lat,lon,elevation,name,country
0,CA008203164,1975-01-01,TMAX,17,45.6,-60.75,15.0,LOWER L'ARDOISE,CA
1,CA008203164,1975-01-01,TMIN,-11,45.6,-60.75,15.0,LOWER L'ARDOISE,CA
2,CA008203164,1975-01-02,TMAX,-11,45.6,-60.75,15.0,LOWER L'ARDOISE,CA
3,CA008203164,1975-01-02,TMIN,-72,45.6,-60.75,15.0,LOWER L'ARDOISE,CA
4,CA008203164,1975-01-03,TMAX,-11,45.6,-60.75,15.0,LOWER L'ARDOISE,CA


Index the dataframe by country, station, date and observation type:

In [11]:
data_1975.set_index(['country','station','date','type'], inplace=True)
data_1975.head()

value   lat    lon  elevation  \
country station     date       type                                  
CA      CA008203164 1975-01-01 TMAX     17  45.6 -60.75       15.0   
                               TMIN    -11  45.6 -60.75       15.0   
                    1975-01-02 TMAX    -11  45.6 -60.75       15.0   
                               TMIN    -72  45.6 -60.75       15.0   
                    1975-01-03 TMAX    -11  45.6 -60.75       15.0   

                                                name  
country station     date       type                   
CA      CA008203164 1975-01-01 TMAX  LOWER L'ARDOISE  
                               TMIN  LOWER L'ARDOISE  
                    1975-01-02 TMAX  LOWER L'ARDOISE  
                               TMIN  LOWER L'ARDOISE  
                    1975-01-03 TMAX  LOWER L'ARDOISE

In [14]:
data_1975.loc['US']

value      lat       lon  elevation  \
station     date       type                                        
USW00024229 1975-01-01 TMAX     83  45.5908  122.6003        5.8   
                       TMIN     17  45.5908  122.6003        5.8   
            1975-01-02 TMAX     78  45.5908  122.6003        5.8   
                       TMIN     17  45.5908  122.6003        5.8   
            1975-01-03 TMAX    100  45.5908  122.6003        5.8   
                       TMIN     50  45.5908  122.6003        5.8   
            1975-01-04 TMAX     83  45.5908  122.6003        5.8   
                       TMIN     56  45.5908  122.6003        5.8   
            1975-01-05 TMAX    133  45.5908  122.6003        5.8   
                       TMIN     50  45.5908  122.6003        5.8   
            1975-01-06 TMAX     67  45.5908  122.6003        5.8   
                       TMIN     44  45.5908  122.6003        5.8   
            1975-01-07 TMAX    111  45.5908  122.6003        5.8   
                       TMIN     33  45.5908  122.6003        5.8   
            1975-01-08 TMAX     83  45.5908  122.6003        5.8   
                       TMIN    -11  45.5908  122.6003        5.8   
            1975-01-09 TMAX     50  45.5908  122.6003        5.8   
                       TMIN      0  45.5908  122.6003        5.8   
            1975-01-10 TMAX     50  45.5908  122.6003        5.8   
                       TMIN     -6  45.5908  122.6003        5.8   
            1975-01-11 TMAX     22  45.5908  122.6003        5.8   
                       TMIN    -22  45.5908  122.6003        5.8   
            1975-01-12 TMAX    100  45.5908  122.6003        5.8   
                       TMIN    -22  45.5908  122.6003        5.8   
            1975-01-13 TMAX    122  45.5908  122.6003        5.8   
                       TMIN     33  45.5908  122.6003        5.8   
            1975-01-14 TMAX    117  45.5908  122.6003        5.8   
                       TMIN     67  45.5908  122.6003        5.8   
            1975-01-15 TMAX     78  45.5908  122.6003        5.8   
                       TMIN     39  45.5908  122.6003        5.8   
...                            ...      ...       ...        ...   
USC00050825 1975-12-19 TMIN   -256  39.1025  107.8989     3002.9   
            1975-12-22 TMAX    -67  39.1025  107.8989     3002.9   
                       TMIN   -233  39.1025  107.8989     3002.9   
            1975-12-23 TMAX    -67  39.1025  107.8989     3002.9   
                       TMIN   -200  39.1025  107.8989     3002.9   
            1975-12-24 TMAX    -78  39.1025  107.8989     3002.9   
                       TMIN   -211  39.1025  107.8989     3002.9   
            1975-12-29 TMAX   -122  39.1025  107.8989     3002.9   
                       TMIN    -67  39.1025  107.8989     3002.9   
            1975-12-31 TMAX   -122  39.1025  107.8989     3002.9   
                       TMIN   -300  39.1025  107.8989     3002.9   
USC00049418 1975-12-02 TMAX    144  38.2000  120.9667      100.9   
                       TMIN     44  38.2000  120.9667      100.9   
            1975-12-03 TMAX    144  38.2000  120.9667      100.9   
                       TMIN     56  38.2000  120.9667      100.9   
            1975-12-04 TMAX    167  38.2000  120.9667      100.9   
                       TMIN     56  38.2000  120.9667      100.9   
            1975-12-05 TMAX    156  38.2000  120.9667      100.9   
                       TMIN     56  38.2000  120.9667      100.9   
            1975-12-08 TMAX    161  38.2000  120.9667      100.9   
                       TMIN     56  38.2000  120.9667      100.9   
            1975-12-09 TMAX    139  38.2000  120.9667      100.9   
                       TMIN     33  38.2000  120.9667      100.9   
            1975-12-10 TMAX     78  38.2000  120.9667      100.9   
                       TMIN     33  38.2000  120.9667      100.9   
            1975-12-11 TMAX     67  38.2000  120.9667      100.9   
                 